# Fetching the Data From a Database for our EDA
Since the csv file 'eda.csv' already existed in the data folder of this repository the following notebook is just serving the purpose to understand how the data was gathered from the database. You do not need to perform the steps from this notebook. They are just for reference and understanding. You can either go through this notebook or go directly check out the notebook [2_EDA.ipnb](2_EDA.ipynb).

This notebook shows how we are connecting to a postgres database in order to fetch data from a schema and create a csv file in order to perform our EDA on it in notebook [2_EDA.ipnb](2_EDA.ipynb)


## Connecting to our database

In the following, we use the package `psycopg2` to connect to a database, retrieve and join data from multiple tables, and export the results to a CSV file for use in this notebook.

### Connecting via psycopg2

In [1]:
import pandas as pd
import psycopg2

import os # provides functions for interacting with operating system
from dotenv import load_dotenv # reads key-value pairs from a .env file and can set them as environment variables

load_dotenv() # takes environment variables from .env

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

We create a connection to the postgres database (using a psycopg2 engine with a connection string containing credentials, host, port, and database name):

In [2]:
# Create / open connection object conn (no need to edit code)
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

## Retrieving data from our database

Before we can get data from the database, we need to create a cursor. A cursor is like a tool that lets Python send commands to the database. We create it using the cursor() method of our connection.

In [3]:
cur = conn.cursor() # create cursor for our opened connection in object conn

### Joining the tables with the correct method

We can ask the database to give us a combined table of houses and their sales history.
To do this, we “link” the house details with the sales records so that every house is included, even if it hasn’t been sold yet.
In Python, we then tell the database to run this request and collect the results so we can work with them in our analysis.

In [4]:
# 1. Define SQL query as a string
query = """
SELECT *
FROM eda.king_county_house_details AS hd
LEFT JOIN eda.king_county_house_sales AS hs
ON hd.id = hs.house_id;
"""

# 2. Execute the query
cur.execute(query)

# 3. Fetch results
data = cur.fetchall()

# 4. Create dataframe and check the head
df = pd.read_sql(query, conn)
df.head()

/var/folders/1h/ym65mz2s4fv2m61786s8zy7m0000gn/T/ipykernel_74652/1066145419.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date,price,house_id,id
0,7129300520,3.0,1.00,1180.0,5650.0,1.0,NaN,0.0,3,7,...,0.0,98178,47.5112,-122.257,1340.0,5650.0,2014-10-13,221900.0,7129300520,1
1,6414100192,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3,7,...,19910.0,98125,47.7210,-122.319,1690.0,7639.0,2014-12-09,538000.0,6414100192,2
2,5631500400,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3,6,...,NaN,98028,47.7379,-122.233,2720.0,8062.0,2015-02-25,180000.0,5631500400,3
3,2487200875,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5,7,...,0.0,98136,47.5208,-122.393,1360.0,5000.0,2014-12-09,604000.0,2487200875,4
4,1954400510,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3,8,...,0.0,98074,47.6168,-122.045,1800.0,7503.0,2015-02-18,510000.0,1954400510,5


### Connection Closing and CSV Creation 

With `conn.close()` we can close the connection to our database again.

In [5]:
#close the connection
conn.close()

In [6]:
# convert the fetched data in a csv file
df.to_csv('data/eda.csv',index=False)

Continue with [2_EDA.ipynb](2_EDA.ipynb).

## Optional Checks in DBeaver

In the following, the queries are listed to ensure that we are not missing any information with the left join query from above

Check for houses without any sales transaction record

SELECT COUNT(*) AS houses_without_sales
FROM king_county_house_details hd
LEFT JOIN king_county_house_sales hs
  ON hd.id = hs.house_id
WHERE hs.house_id IS NULL;

*Gives a table with houses_without_sales = 0 as output*

Give an overview of houses with or without sales

SELECT
  COUNT(*) AS total_houses,
  COUNT(hs.house_id) AS houses_with_sales,
  COUNT(*) - COUNT(hs.house_id) AS houses_without_sales
FROM king_county_house_details hd
LEFT JOIN king_county_house_sales hs
  ON hd.id = hs.house_id;

*Gives us a table where* 
| total_houses     | houses_with_sales     | houses_without_sales     |
|-------------|-------------|-------------|
| 21597 | 21597 | 0 |